# Updating game_stats table through previous day
This notebook updates the game_stats table in our nba_dfs_model database. These are the traditional game stats, plus usage.<br>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import sqlite3
import sys
import os

yesterday_only = None
#site = None

# Get the parent directory where config.py is located
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

API_KEY = None
API_HOST = None

# if API_KEY is None or API_HOST is None:
#     raise ValueError("API_KEY and API_HOST must be provided when running via Papermill.")

In [2]:
# ✅ Call datetime.now() only once
now = datetime.now()

# ✅ Define date variables
today = now.strftime('%Y%m%d')
yesterday = (now - timedelta(days=1)).strftime('%Y%m%d')
last_day = (now - timedelta(days=2)).strftime('%Y%m%d')

In [3]:
today, yesterday, last_day

('20250210', '20250209', '20250208')

In [4]:
from config import API_KEY, API_HOST

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": API_HOST
}

The following cell checks that we have game_stats up until two days ago, so that we only have to update the previous day's stats. If the yesterday_only variable is True, then we only need the previous day.

In [5]:
# Connect to your SQLite database
conn = sqlite3.connect('../nba_dfs_model.db')
# 
# Use parameterized query to prevent SQL injection
query = "SELECT DISTINCT game_id FROM game_stats ORDER BY game_id DESC LIMIT 1"
cursor = conn.cursor()
cursor.execute(query)

# Fetch the results
results = cursor.fetchall()

# Print the results
for row in results:
    if row[0][:8] == yesterday:
        print("game_stats table is already up to date.")
        sys.exit(0)
    elif row[0][:8] == last_day:
        yesterday_only = True
    else: 
        yesterday_only = False
        last_day = row[0][:8] #datetime.strptime(row[0][:8], '%Y%m%d')
        

print(f"yesterday_only value: {yesterday_only}")

# # Close the connection
# conn.close()
# # with sqlite3.connect("nba_dfs_model.db") as conn:
# #     cursor = conn.cursor()

# #     # Query to count the number of rows in the table
# #     cursor.execute("SELECT * FROM game_stats WHERE game_id IN (?, ?);")
# #     query = cursor.fetchall()

# #     print(query)


yesterday_only value: True


If we only need to update with the previous day's stats, the if block of this cell runs. If not, the else block runs beginning with the first day that needs to be updated going through the previous day.

In [6]:
#This code tests to see if the most recent games in the table are from two days ago
#If not, it checks for the latest day of games, and retrieves game_ids for all the necessary games
#The else part of this statement hasn't been tested as of 12/17/2024

game_ids = []
no_game_dates = []
if yesterday_only:
        
    url = f"https://tank01-fantasy-stats.p.rapidapi.com/getNBAGamesForDate?gameDate={yesterday}"
    
    headers = {
        "x-rapidapi-key": API_KEY,
        "x-rapidapi-host": API_HOST
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for HTTP codes 4xx/5xx
        result = response.json()
        
        if 'body' in result and result['body']:
            for game in result['body']:
                game_ids.append(game['gameID'])
        else:
            no_game_dates.append(current_date)  # Track dates with no games
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {current_date}: {e}")
    except KeyError:
        print(f"Unexpected response format for {current_date}: {result}")

else:
    start_date = datetime.strptime(last_day, '%Y%m%d')
    end_date = datetime.strptime(today, '%Y%m%d')
    delta = end_date - start_date
    diff = delta.days
    games_added = 0     # Counter for games added
    game_ids = []

# Loop through each day
    for i in range(1, diff):
        day_diff = timedelta(days=i)
        current_date = (start_date + day_diff).strftime('%Y%m%d')
        
        url = f"https://tank01-fantasy-stats.p.rapidapi.com/getNBAGamesForDate?gameDate={current_date}"
        
        # headers = {
        #     "x-rapidapi-key": "3103a75392msh7bce7c32fde122cp134393jsn4d42ed6d08a8",
        #     "x-rapidapi-host": "tank01-fantasy-stats.p.rapidapi.com"
        # }
        
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an error for HTTP codes 4xx/5xx
            result = response.json()
            
            if 'body' in result and result['body']:
                for game in result['body']:
                    game_ids.append(game['gameID'])                  
            else:
                no_game_dates.append(current_date)  # Track dates with no games
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {current_date}: {e}")
        except KeyError:
            print(f"Unexpected response format for {current_date}: {result}")

In [7]:
#Connect to the database
conn = sqlite3.connect("../nba_dfs_model.db")
cursor = conn.cursor()

# Execute the query
query = "SELECT * FROM game_stats ORDER BY game_id DESC LIMIT 5" 
#date = '20231115'  # Replace with your desired date
cursor.execute(query)

# Fetch results
results = cursor.fetchall()
print(results)

# game_ids = []

# # Print results
# for result in results:
#     game_ids.append(result[0])

# Close the cursor and connection
cursor.close()
conn.close()

[('Brice Sensabaugh', '20250208_UTA@LAC', '949747621869', '29', 'UTA', 'UTA', 7, 0, 0, 3, 0, 3, 0, 0, 0, 2, '0', 2, 6, 2, 3, 30.91, 14), ('Johnny Juzang', '20250208_UTA@LAC', '94934219027', '29', 'UTA', 'UTA', 11, 1, 4, 7, 2, 6, 2, 1, 0, 2, '-6', 3, 19, 4, 1, 22.29, 25), ('Kyle Filipowski', '20250208_UTA@LAC', '948347491869', '29', 'UTA', 'UTA', 7, 5, 1, 5, 3, 2, 4, 2, 0, 4, '0', 0, 13, 5, 2, 15.4, 29), ('Isaiah Collier', '20250208_UTA@LAC', '948240427989', '29', 'UTA', 'UTA', 5, 9, 0, 1, 4, 2, 1, 3, 2, 4, '+1', 0, 5, 3, 2, 13.07, 29), ('MarJon Beauchamp', '20250208_UTA@LAC', '947644156669', '13', 'LAC', 'LAC', 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, '+3', 0, 3, 0, 0, 21.38, 2)]


In [8]:
game_ids

['20250209_TOR@HOU', '20250209_PHI@MIL', '20250209_CHA@DET']

In [9]:
len(game_ids)

3

In [10]:
# Connect to the SQLite database
# Using the game_ids to add game data points
with sqlite3.connect("../nba_dfs_model.db") as conn:
    cursor = conn.cursor()

# Counter for games processed
games_processed = 0
rows_added = 0  # Initialize the counter for rows added

# Main loop to process game IDs
for game_id in game_ids:
    try:
        # Make the API request
        url = f"https://tank01-fantasy-stats.p.rapidapi.com/getNBABoxScore?gameID={game_id}"
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        result = response.json()

        # Skip games with no player stats
        if 'body' not in result or 'playerStats' not in result['body']:
            print(f"No player stats found for game ID {game_id}. Skipping...")
            continue

        current_dict = result['body']['playerStats']

        # Insert player stats into the database
        for player_id, stats in current_dict.items():
            try:
                cursor.execute('''
                INSERT OR REPLACE INTO game_stats (
                    longName, game_id, player_id, team_id, team, teamAbv, fga, ast, tptfgm, fgm, fta, tptfga,
                    OffReb, ftm, blk, DefReb, plusMinus, stl, pts, PF, TOV, usage, mins
                ) VALUES (
                    ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?
                )
                ''', (
                    stats.get('longName', ''), stats['gameID'], stats['playerID'],  stats.get('teamID', ''), 
                    stats.get('team', ''), stats.get('teamAbv', ''), stats.get('fga', 0), stats.get('ast', 0),
                    stats.get('tptfgm', 0), stats.get('fgm', 0), stats.get('fta', 0), stats.get('tptfga', 0),
                    stats.get('OffReb', 0), stats.get('ftm', 0), stats.get('blk', 0), stats.get('DefReb', 0), 
                    stats.get('plusMinus', ''), stats.get('stl', 0), stats.get('pts', 0), stats.get('PF', 0), 
                    stats.get('TOV', 0), stats.get('usage', 0.0), stats.get('mins', 0)
                ))
                rows_added += 1
                if rows_added % 1000 == 0:
                    print(f"Progress update: {rows_added} rows added so far.")
            except KeyError as e:
                print(f"KeyError: {e} for player {player_id} in game {game_id}")
            except sqlite3.Error as e:
                print(f"SQL Error: {e}")

        conn.commit()

        # Update games processed
        games_processed += 1
        if games_processed % 100 == 0:
            print(f"Progress update: {games_processed} games processed so far.")

    except requests.exceptions.RequestException as e:
        print(f"API request failed for game ID {game_id}: {e}")
    except sqlite3.Error as e:
        print(f"Database error: {e}")

# Final summary
print(f"Database updated successfully with {rows_added} rows added from {games_processed} games.")


Database updated successfully with 59 rows added from 3 games.


Finding number of rows in the game_stats table

In [11]:
# Connect to the database
conn = sqlite3.connect("../nba_dfs_model.db")
cursor = conn.cursor()

# Execute the query to get the row count
query = "SELECT COUNT(*) FROM game_stats"
cursor.execute(query)

# Fetch the result
num_rows = cursor.fetchone()[0]
print(f"There are {num_rows} rows in the game_stats table.")

# Close the connection
conn.close()



There are 53555 rows in the game_stats table.
